## Description du notebook

Ce notebook a pour but de vous permettre de jouer avec des fonctions, où votre objectif sera de comprendre:
- la représentation graphique de fonctions mathématiques et les notions d'input et d'output
- la représentation des fonctions linéaires et affines en particulier
- la notion de dérivée d'une fonction en un point, et sa géométrie sous-jacente
- l'intuiton géométrique de la régression linéaire univariée
- la notion de fonction de coût

Par ailleurs, le code Python fourni, qui utilise ipywidgets et ipycanvas, peut vous servir de sujet d'exploration pour comprendre comment s'élabore un petit logiciel graphique en Python. C'est déjà un bon niveau de structure de code à aborder, et différents usage de la syntaxe Python ici pourront vous donner des idées. N'hésitez pas à bidouiller le code pour voir ce qu'il se passe ! Si vous cassez trop de choses, vous pouvez toujours le télécharger de nouveau.

Dans cette première section, vous avez un repère pour vous aider à représenter une fonction f (par défaut la fonction `cos` de la librairie mathématique standarde de python, `math`). Changez la fonction f, jouez avec les sliders.

Fonctions importantes à connaître et savoir reconnaître, à titre d'exemple: 
- `math.sin(x)`
- `math.tan(x)`
- `math.exp(x)`
- `math.log(x)`
- `math.sqrt(x)`
- `C`, où C est un nombre fixe, par exemple 7
- `x`
- `-x`
- `x*x`
- `x*x*x`
- `1 / x`
- `1 / (1 + math.exp(x))`
- des combinaisons d'opération, par exemple `7 * math.cos(4*x - 3) - 2`

In [ ]:
#pip3 install ipywidgets
#pip3 install ipycanvas

import math
import ipywidgets as widgets
from ipycanvas import MultiCanvas #https://ipycanvas.readthedocs.io/en/latest/api.html

width = 600
height = 400
step = 1 #increment of the coordinate frame
scale = 20 #number of pixels corresponding to a step
mcanvas = MultiCanvas(3, width = width, height = height)
canvas = mcanvas[0]
fcanvas = mcanvas[1]
fcanvas.fill_style = "rgba(0, 0, 0, 0)"
fcanvas.fill_rect(0, 0, width, height)
pcanvas = mcanvas[2]

f = lambda x : math.cosh(x) #the function to display

#change x or y from canvas coordinates to vector space coordinates
def normalize(x, translate):
    return (x - translate) * step / scale

#change x or y from vector space coordinates to canvas coordinates
def denormalize(x, translate):
    return (x * scale) / step + translate

#a class for a 2d vector object
class Vec2:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __str__(self):
        return "(" + str(self.x) + ", " + str(self.y) + ")"
    
    def __add__(self,other):
        if (not isinstance(other, Vec2)):
            print("Invalid vector __add__")
            raise ValueError
        res = Vec2(self.x + other.x, self.y + other.y)
        return res

    def __sub__(self,other):
        if (not isinstance(other, Vec2)):
            print("Invalid vector __sub__")
            raise ValueError
        res = Vec2(self.x - other.x, self.y - other.y)
        return res

    #scalar multiplication for float-vector
    #schur multiplication, not complex multiplication, for vector-vector
    def __mul__(self,other):
        if (isinstance(other, int) or isinstance(other, float)):
            res = Vec2(self.x * other, self.y * other)
            return res
        elif (not isinstance(other, Vec2)):
            print("Invalid vector __mul__")
            raise ValueError
        res = Vec2(self.x * other.x, self.y * other.y)
        return res  

    def __div__(self,other):
        if (not isinstance(other, Vec2)):
            print("Invalid vector __div__")
            raise ValueError
        res = Vec2(self.x / other.x, self.y / other.y)
        return res

def draw_coordinate_frame(color="black"):
    canvas.stroke_style = "black"
    notch_size = 4
    center_x = width / 2
    center_y = height / 2
    
    #borders
    canvas.stroke_rect(0, 0, width, height)
    
    #axes
    canvas.move_to(       0, center_y)
    canvas.line_to(   width, center_y)
    canvas.move_to(center_x,        0)
    canvas.line_to(center_x,   height)
    canvas.stroke()
    
    #top notches
    canvas.move_to(center_x, center_y)
    i = center_y
    j = 1
    while i >= 0:
        canvas.move_to(center_x - notch_size, i)
        canvas.line_to(center_x + notch_size, i)
        canvas.stroke_text(str(j * step), center_x + 2 * notch_size, i - scale * 0.8)
        i -= scale
        j += 1

    #bottom notches
    canvas.move_to(center_x, center_y)
    i = center_y
    j = 1
    while i <= height:
        canvas.move_to(center_x - notch_size, i)
        canvas.stroke_text(str(-j * step), center_x + 2 * notch_size, i + scale * 1.20)
        canvas.line_to(center_x + notch_size, i)
        i += scale
        j += 1

    #left notches
    canvas.move_to(center_x, center_y)
    i = center_x
    j = 1
    while i >= 0:
        canvas.move_to(i, center_y - notch_size)
        canvas.stroke_text(str(-j * step), i - scale * 1.25, center_y - 2 * notch_size)
        canvas.line_to(i, center_y + notch_size)
        i -= scale
        j += 1

    #right notches
    canvas.move_to(center_x, center_y)
    i = center_x
    j = 1
    while i <= width:
        canvas.move_to(i, center_y - notch_size)
        canvas.stroke_text(str(j * step), i + 0.8 * scale, center_y - 2 * notch_size)
        canvas.line_to(i, center_y + notch_size)
        i += scale
        j += 1
    canvas.stroke()

def draw_function_curve(f):
    fcanvas.stroke_style = "red"
    inputs = [normalize(i, width/2) for i in range(width)] #(math.ceil(width/2 + 1), width)]
    outputs = list(map(f, inputs))
    fcanvas.move_to(denormalize(inputs[0], width/2), denormalize(-outputs[0], height/2))
    for i in range(1, len(inputs)):
        x = inputs[i]
        f_x = outputs[i]
        fcanvas.line_to(denormalize(x, width/2), denormalize(-f_x, height/2))
    fcanvas.stroke()

def draw_point(arg1, arg2=None, radius=2):
    if arg2 == None:
        if not isinstance(arg1, Vec2):
            print("draw_point: Invalid point")
            raise ValueError
        pcanvas.fill_arc(arg1.x, arg1.y, radius, 0, 360)
    else:
        pcanvas.fill_arc(arg1, arg2, radius, 0, 360)
    pcanvas.stroke()

def draw_vector(vec1, vec2=None, color="black"):
    pcanvas.stroke_style=color
    if vec2 == None:
        start_point = Vec2(denormalize(     0, width/2), denormalize(     0, height/2))
        end_point   = Vec2(denormalize(vec1.x, width/2), denormalize(-vec1.y, height/2))
    else:
        start_point = Vec2(denormalize(vec1.x, width/2), denormalize(-vec1.y, height/2))
        end_point   = Vec2(denormalize(vec2.y, width/2), denormalize(-vec2.y, height/2))
    pcanvas.move_to(start_point.x, start_point.y)
    pcanvas.line_to(  end_point.x,   end_point.y)
    pcanvas.stroke()
    pcanvas.fill_arc(end_point.x, end_point.y, 3, 0, 360)
    pcanvas.stroke()


#canvas clear and redraw button
#redraw_button = widgets.Button(
#    description='Clear points',
#    disabled=False,
#    button_style='', # 'success', 'info', 'warning', 'danger' or ''
#    tooltip='Redraw the graph',
#    icon='check'
#)
#def on_button_clicked(b):
#    pcanvas.clear()
#    draw_coordinate_frame()
#    draw_function_curve(f)
#redraw_button.on_click(on_button_clicked)


#x value slider widget
x = 0.1
f_x = f(x)
x_slider = widgets.FloatSlider(
    value=x,
    min=normalize(0, width/2),
    max=normalize(width, width/2),
    step=0.1,
    description='Value of x:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
def on_x_value_change(change):
    pcanvas.clear()
    global x
    global f_x
    x = change['new']
    f_x = f(x)
    draw_point(denormalize(x, width/2), denormalize(-f_x, height/2))
    pcanvas.fill_text(f"{x}   -f->   {f_x}", width - 150, 20)
x_slider.observe(on_x_value_change, names='value')

display(x_slider)
#display(redraw_button)

draw_coordinate_frame()
draw_function_curve(f)

draw_vector(Vec2(1, 3.5))

mcanvas

Dans la section suivante, on s'intéresse à présent aux fonctions linéaires et aux fonctions affines.
Les fonctions linéaires sont les fonctions de la forme `x -> a*x`, les fonctions affines sont les fonctions de la forme `x -> a*x + b`, où `a` et `b` sont des nombres constants choisis (qualifiés de "paramètres").

Jouez avec les sliders.
- Quelle est la particularité des fonctions linéaires et des fonctions affines par rapport à la plupart des fonctions de l'exercice précédent ?
- Quelle est la particularité des fonctions linéaires par rapport aux fonctions affines ?
- Que fait `a` ?
- Que fait `b` ?
- Quels sont les valeurs remarquables de `a` et de `b` ?

In [ ]:
canvas.clear()
fcanvas.clear()
pcanvas.clear()
draw_coordinate_frame()

mcanvas

a = 1.0
b = 0.0

affine_func = lambda x : a*x + b

def draw_affine(aff_func, color="green", layer=fcanvas):
    layer.stroke_style = color
    layer.begin_path()
    layer.clear_rect(0, 0, width, height)
    layer.stroke()
    layer.move_to(    0, denormalize(-aff_func(normalize(    0, width/2)), height/2))
    layer.line_to(width, denormalize(-aff_func(normalize(width, width/2)), height/2))
    layer.close_path()
    layer.stroke()
    mcanvas

slope_slider = widgets.FloatSlider(
    value             = a,
    min               = -30,
    max               = 30,
    step              = 0.01,
    description       = 'Value of a:',
    disabled          = False,
    continuous_update = True,
    orientation       = 'horizontal',
    readout           = True,
    readout_format    = '.2f',
)
const_slider = widgets.FloatSlider(
    value             = b,
    min               = normalize(0, width/2),
    max               = normalize(width, width/2),
    step              = 0.01,
    description       = 'Value of b:',
    disabled          = False,
    continuous_update = True,
    orientation       = 'horizontal',
    readout           = True,
    readout_format    = '.2f',
)
def on_slope_change(change):
    global a
    a = change['new']
    affine_func = lambda x: a*x + b
    draw_affine(affine_func)
def on_const_change(change):
    global b
    b = change['new']
    affine_func = lambda x: a*x + b
    draw_affine(affine_func)
slope_slider.observe(on_slope_change, names='value')
const_slider.observe(on_const_change, names='value')
display(slope_slider)
display(const_slider)


affine_func = lambda x: a*x + b
draw_affine(affine_func)

mcanvas

Dans cette section, on va essayer de comprendre la notion de dérivée en un point. La dérivée en un point est le taux d'accroissement entre un point et un autre point très, très proche. Cela donne une approximation linéaire du comportement de la fonction en ce point.
La dérivée est essentielle pour comprendre comment on trouve des solutions linéaires à des problèmes non-linéaires.

Jouez avec les sliders, le point `x` correspond au point en lequel on veut trouver la dérivée, le point `x+h` est un point qui doit être juste à côté: plus `h` est petit, plus l'écart entre `x` et `x+h` est faible, et plus le taux d'accroissement de `f(x)` à `f(x+h)` s'approche de la dérivée.

La dérivée en un point `x` est la pente (le nombre `a`) de la fonction affine passant par ce point `x` et tangente à la courbe de `f` en `x`.

In [ ]:
fcanvas.begin_path()
fcanvas.clear_rect(0, 0, width, height)
fcanvas.close_path()
pcanvas.clear_rect(0, 0, width, height)

mcanvas

#x+h value slider widget
x_plus_h = 0.0
f_x_plus_h = f(x_plus_h)

def calculate_slope():    
    global x_plus_h
    global f_x_plus_h
    if x == x_plus_h:
        x_plus_h += 0.0001
        f_x_plus_h = f(x_plus_h)
    return (f_x_plus_h - f_x) / (x_plus_h - x)

x_plus_h_slider = widgets.FloatSlider(
    value             = 0.0,
    min               = normalize(0, width/2),
    max               = normalize(width, width/2),
    step              = 0.1,
    description       = 'Value of x+h:',
    disabled          = False,
    continuous_update = True,
    orientation       = 'horizontal',
    readout           = True,
    readout_format    = '.1f',
)

def on_x_plus_h_value_change(change):
    pcanvas.clear()
    global x_plus_h
    global f_x_plus_h
    x_plus_h    = change['new']
    f_x_plus_h  = f(x_plus_h)
    a           = calculate_slope()
    b           = f_x - a*x 
    affine_func = lambda x: a*x + b
    draw_affine(affine_func, layer = pcanvas)
    draw_point(denormalize(x       , width/2), denormalize(-f_x       , height/2), radius = 3)
    draw_point(denormalize(x_plus_h, width/2), denormalize(-f_x_plus_h, height/2), radius = 3)
    mcanvas
x_plus_h_slider.observe(on_x_plus_h_value_change, names = 'value')

display(x_slider)
display(x_plus_h_slider)

#Calculate cost function button
slope_button = widgets.Button(
    description  = 'Calculate slope',
    disabled     = False,
    button_style = '', # 'success', 'info', 'warning', 'danger' or ''
    tooltip      = 'Calculate the value of the slope at for current x and h values',
)
def on_slope_button_clicked(b):
    slope = calculate_slope()
    pcanvas.clear_rect(width - 150, height - 90, width, height)
    pcanvas.fill_text(f"Slope 'a': {slope}", width - 150, height - 50)
    pcanvas.stroke()
slope_button.on_click(on_slope_button_clicked)
display(slope_button)

draw_coordinate_frame()
draw_function_curve(f)

mcanvas


Dans cette dernière section, on va tenter de comprendre l'intuition des mathématiques derrière la régression linéaire - le problème le plus standard d'optimisation de modèle statistique.

La fonction de coût fournie (appelée "Mean Squared Error", ou "erreur au carré moyenne"), permet de mesurer l'écart entre le modèle (la fonction affine choisie) et sa capacité à prédire les points. Cette fonction est toujours positive. Pour avoir un modèle efficace, il faut chercher à minimiser la fonction de coût.

- Quel genre de valeurs de `ax + b` donnent de bons résultats ? À quoi cela correspond-il, géométriquement parlant ?
- Quel méthode conseilleriez-vous pour minimiser la fonction de coût ?

In [ ]:
fcanvas.clear()
fcanvas.stroke()
pcanvas.clear()
pcanvas.stroke()

from random import random

#MSE: mean squared error
def cost_function(data_points, model_func):
    real_inputs   = list(map(lambda point: point.x, data_points))
    real_outputs  = list(map(lambda point: point.y, data_points))
    model_outputs = list(map(lambda x: model_func(x), real_inputs))
    result = 0
    for i in range(len(model_outputs)):
        prediction   = model_outputs[i]
        expectation  = real_outputs[i]
        distance     = prediction - expectation
        error        = distance * distance
        result      += error
    return result / len(model_outputs)
    
def get_random_point_list(nb_of_points, x_min=-15, x_max=15, y_min=-5, y_max=8):
    result = []
    for i in range(nb_of_points):
        x = random() * (x_max - x_min) + x_min
        y = random() * (y_max - y_min) + y_min
        result += [Vec2(x, y)]
    return result

data_points = get_random_point_list(15, x_max=0, y_max=3)
data_points += get_random_point_list(15, x_min=-3, y_min=-2)

for point in data_points:
    draw_point(
        denormalize( point.x,  width/2),
        denormalize(-point.y, height/2)
    )

cost = cost_function(data_points, affine_func)
    
def update_cost(affine_func):
    global cost
    cost = cost_function(data_points, affine_func)

def on_slope_change(change):
    global a
    a = change['new']
    affine_func = lambda x: a*x + b
    draw_affine(affine_func)
    update_cost(affine_func)

def on_const_change(change):
    global b
    b = change['new']
    affine_func = lambda x: a*x + b
    draw_affine(affine_func)
    update_cost(affine_func)

slope_slider.min = -3
slope_slider.max = 3
slope_slider.observe(on_slope_change, names='value')
const_slider.observe(on_const_change, names='value')
display(slope_slider)
display(const_slider)

#Calculate cost button
cost_button = widgets.Button(
    description  = 'Calculate cost',
    disabled     = False,
    button_style = '', # 'success', 'info', 'warning', 'danger' or ''
    tooltip      = 'Calculate the cost function for the current linear model',
)
def on_cost_button_clicked(b):
    update_cost(affine_func)
    pcanvas.clear_rect(width-150, height-90, width, height)
    pcanvas.fill_text(f"Cost: {cost}", width-150, height-50)
    pcanvas.stroke()

cost_button.on_click(on_cost_button_clicked)
display(cost_button)

draw_coordinate_frame()
draw_function_curve(f)

mcanvas
